# 02. Preprocessing - Data Transformation


## Experimenting

In [2]:
from jupyter_init import setup

setup()

from src_code.config import *

REPOS = []


In [7]:
# transformed_df = transform(repos_filter=None if not REPOS else REPOS, workers=8)
TRANSFORMED_DF = DEFECTORS_DIR / "jit_bug_prediction_splits" / "time" / "train_labeled_features_partial_copy.feather"

# load transformed dataframe and examine
import pandas as pd
df = pd.read_feather(TRANSFORMED_DF)
# print(df['has_bug_kw'].head())
# print(df['label'].head())
print(df['label'].value_counts(normalize=True))
print(df['repo'].value_counts(normalize=True))
print(f"Total rows in transformed dataframe: {len(df)}")

#null_counts = df.isnull().sum()
#print("Null value counts per column:")
print(df.isnull().sum())


print(df.columns)
print(df.describe())
# print(f"Total columns in transformed dataframe: {len(df.columns)}")
# print(f"Total rows in transformed dataframe: {len(df)}")



label
1    0.529703
0    0.470297
Name: proportion, dtype: float64
repo
airflow    1.0
Name: proportion, dtype: float64
Total rows in transformed dataframe: 202
datetime                      0
commit                        0
repo                          0
filepath                      0
content                       0
methods                       0
lines                         0
author_email                  0
canonical_datetime            0
author_exp_pre                0
author_recent_activity_pre    0
label                         0
loc_added                     0
loc_deleted                   0
files_changed                 0
hunks_count                   0
msg_len                       0
has_fix_kw                    0
has_bug_kw                    0
ast_delta                     0
complexity_delta              0
max_func_change               0
time_since_last_change        0
todo                          0
fixme                         0
try                           0
except 

In [21]:
import pandas as pd

# Assume TRANSFORMED_DF is defined and loaded as df
# df = pd.read_feather(TRANSFORMED_DF) # Already loaded in user context

print("## ✨ Statistical Summary of New Feature Columns\n")

# Identify the newly added columns (everything after 'label')
new_feature_cols = [
    'loc_added', 'loc_deleted', 'files_changed', 'hunks_count',
    'msg_len', 'has_fix_kw', 'has_bug_kw', 
    'ast_delta', 'complexity_delta', 'max_func_change', 
    'author_exp_pre', 'author_recent_activity_pre', 'time_since_last_change', 
    'todo', 'fixme', 'try', 'except', 'raise'
]

# 1. Summary Statistics for Numeric Columns
print("### 1. Summary Statistics (Mean, Median, Max)\n")
# Filter out embedding lists/None values and analyze numeric columns
numeric_df = df[new_feature_cols].describe().T[['count', 'mean', '50%', 'max']].rename(columns={'50%': 'median'})
print(numeric_df.to_markdown(numalign="left", stralign="left"))

print("\n" + "---" + "\n")

# 2. Value Counts for Flag/Keyword Columns
print("### 2. Value Counts for Flag Features\n")
flag_cols = ['has_fix_kw', 'has_bug_kw']
for col in flag_cols:
    print(f"**{col.upper()}**:")
    print(df[col].value_counts(normalize=True).to_frame().rename(columns={'proportion': 'Count Percentage'}).to_markdown(numalign="left", stralign="left"))
    print("\n")

print("\n" + "---" + "\n")

# 3. Embedding Status Check
print("### 3. Embedding Status\n")
print(f"Code Embeddings (None/List): {df['code_embed'].apply(lambda x: 1 if x is not None else 0).value_counts()}")

## ✨ Statistical Summary of New Feature Columns

### 1. Summary Statistics (Mean, Median, Max)

|                            | count   | mean    | median   | max         |
|:---------------------------|:--------|:--------|:---------|:------------|
| loc_added                  | 1000    | 79.706  | 23       | 1378        |
| loc_deleted                | 1000    | 111.201 | 54       | 1893        |
| files_changed              | 1000    | 4.924   | 4        | 19          |
| hunks_count                | 1000    | 31.282  | 18       | 862         |
| msg_len                    | 1000    | 71.996  | 59       | 290         |
| has_fix_kw                 | 1000    | 0.127   | 0        | 1           |
| has_bug_kw                 | 1000    | 0.2     | 0        | 1           |
| ast_delta                  | 1000    | 73.593  | 0        | 837         |
| complexity_delta           | 1000    | 2.711   | 0        | 40          |
| max_func_change            | 1000    | 36.419  | 36       | 205   